In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
import tensorflow as tf
print(tf.__version__)

# !pip install -q tensorflow-datasets

2.4.1


In [2]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteJQ1H2X/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteJQ1H2X/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteJQ1H2X/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# train_data와 test_data를 위의 4개의 배열로 복사 (sentence와 label 각각 따로 append)

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())
  
# 추후 모델 학습을 위해 numpy array로 변환
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [4]:
# encoding sentences to sequences

vocab_size = 10000 # 전체 단어들 index화 될 수 있는 최대 개수 (by. commonality)
embedding_dim = 16
max_length = 120 # 문장 encoding 후 최대 길이 (문장에 속한 단어들의 최대 개수 : 최대 120단어)
trunc_type='post' # 뒤에서부터 잘림 (truncate)
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type) # padding & trunc로 모두 120 길이!

# training sentences의 word index 기반의 tokenizer로 encoding! (sentences to sequences)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences) 
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [11]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text]) # get(찾고자하는 값의 키, 없을 때 출력할 값)

print('- padded[1] :', padded[1])
print('- decoded padded[1] :', decode_review(padded[1])) # ??? 들 : padding 결과인 0 이라는 키에 대한 value가 없으므로! ()
print('- training ori sentence', training_sentences[1])

- padded[1] : [   0    0    0    0    0    0    0    0   11   26   75  571    6  805
 2354  313  106   19   12    7  629  686    6    4 2219    5  181  584
   64 1454  110 2263    3 3951   21    2    1    3  258   41 4677    4
  174  188   21   12 4078   11 1578 2354   86    2   20   14 1907    2
  112  940   14 1811 1340  548    3  355  181  466    6  591   19   17
   55 1817    5   49   14 4044   96   40  136   11  972   11  201   26
 1046  171    5    2   20   19   11  294    2 2155    5   10    3  283
   41  466    6  591    5   92  203    1  207   99  145 4382   16  230
  332   11 2486  384   12   20   31   30]
- decoded padded[1] : ? ? ? ? ? ? ? ? i have been known to fall asleep during films but this is usually due to a combination of things including really tired being warm and comfortable on the <OOV> and having just eaten a lot however on this occasion i fell asleep because the film was rubbish the plot development was constant constantly slow and boring things seemed to happ

In [17]:
print(reverse_word_index[1])
print(reverse_word_index[2])
# print(reverse_word_index[0]) ==> index error! (키 값 1부터 시작하므로 0은 없음. 0은 padding 결과물)

<OOV>
the


In [6]:
# 모델 구성

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length), # 1000, 16, 120
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [7]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 7s 6ms/step - loss: 0.6005 - accuracy: 0.6466 - val_loss: 0.3412 - val_accuracy: 0.8517
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2429 - accuracy: 0.9079 - val_loss: 0.3665 - val_accuracy: 0.8423
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0925 - accuracy: 0.9783 - val_loss: 0.4559 - val_accuracy: 0.8284
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0219 - accuracy: 0.9977 - val_loss: 0.5316 - val_accuracy: 0.8271
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0060 - accuracy: 0.9997 - val_loss: 0.5941 - val_accuracy: 0.8282
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.6460 - val_accuracy: 0.8282
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 9.3146e-04 - accuracy: 1.0000 - val_loss: 0.6907 - val_accuracy: 0.8288
Ep

In [8]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [9]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [10]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
